# features extracted by original file

In [1]:
import feature_extractor_xs as ft
from PIL import Image
import numpy
import os

# Instantiate extractor. Slow if called first time on your machine, as it
# needs to download 100 MB.
extractor = ft.YouTube8MFeatureExtractor()

image_file = os.path.join(extractor._model_dir, 'cropped_panda.jpg')

im = numpy.array(Image.open(image_file))

pre_f, frame_features_pca, frame_features_pca_qt = extractor.extract_rgb_frame_features(im)



/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
im.shape

(100, 100, 3)

In [4]:
import numpy as np
print(pre_f.shape)
pre_f
# (pre_f-pca_mean).reshape((1,2048)).dot(pca_eigenvecs).reshape((1024,))/(np.sqrt(pca_eigenvals + 1e-4))



(2048,)


array([0.19233467, 0.04197577, 0.14644754, ..., 0.11446106, 0.01030327,
       0.1371786 ], dtype=float32)

In [5]:
frame_features_pca

array([-2.051016  ,  1.4200372 , -0.47344914, ...,  0.769946  ,
        0.09573443, -1.3794014 ], dtype=float32)

In [6]:
# features = numpy.clip(frame_features_pca, min_quantized_value, max_quantized_value)
# quantize_range = max_quantized_value - min_quantized_value
# features_ = (features - min_quantized_value) * (255.0 / quantize_range)
# features_ = [int(round(f)) for f in features_]
# features_
frame_features_pca_qt

[0,
 218,
 97,
 153,
 171,
 129,
 155,
 146,
 176,
 162,
 197,
 105,
 142,
 114,
 118,
 141,
 132,
 123,
 119,
 135,
 105,
 170,
 130,
 103,
 152,
 133,
 142,
 145,
 150,
 140,
 110,
 118,
 118,
 139,
 157,
 123,
 142,
 115,
 81,
 152,
 147,
 148,
 60,
 156,
 124,
 105,
 155,
 111,
 120,
 96,
 129,
 186,
 89,
 117,
 105,
 107,
 134,
 150,
 79,
 126,
 133,
 75,
 101,
 75,
 137,
 139,
 69,
 131,
 85,
 118,
 133,
 89,
 227,
 138,
 83,
 91,
 130,
 130,
 164,
 166,
 162,
 116,
 119,
 173,
 199,
 169,
 100,
 107,
 152,
 172,
 93,
 92,
 142,
 129,
 102,
 96,
 93,
 131,
 116,
 62,
 109,
 102,
 159,
 86,
 103,
 164,
 91,
 126,
 110,
 71,
 105,
 165,
 159,
 139,
 60,
 170,
 175,
 0,
 94,
 116,
 200,
 154,
 222,
 125,
 107,
 125,
 101,
 62,
 125,
 145,
 108,
 95,
 220,
 204,
 140,
 126,
 85,
 51,
 168,
 0,
 172,
 139,
 145,
 180,
 231,
 53,
 139,
 166,
 123,
 130,
 167,
 172,
 101,
 151,
 86,
 103,
 150,
 101,
 194,
 148,
 252,
 107,
 192,
 134,
 146,
 169,
 105,
 100,
 28,
 229,
 129,
 101,
 148

# merge two graph: classify_image_graph_def.pb and model/pca_qt_model.pb

In [7]:
import tensorflow as tf
import numpy
import os

tf.reset_default_graph()
# input: [300, 2048]
# output: [300, 2014]
MODEL_DIR = os.path.join(os.getenv('HOME'), 'yt8m')
_model_dir = MODEL_DIR
batch_size = 1
# graph = tf.Graph()
# with graph.as_default():
tf.reset_default_graph()
def _load_pca():
    pca_mean = numpy.load(
        os.path.join(_model_dir, 'mean.npy'))[:, 0]
    pca_eigenvals = numpy.load(
        os.path.join(_model_dir, 'eigenvals.npy'))[:1024, 0]
    pca_eigenvecs = numpy.load(
        os.path.join(_model_dir, 'eigenvecs.npy')).T[:, :1024]
    return pca_mean, pca_eigenvals, pca_eigenvecs

def _make_bytes(int_array):
    if bytes == str:  # Python2
        return ''.join(map(chr, int_array))
    else:
        return bytes(int_array)

def quantize(features, min_quantized_value=-2.0, max_quantized_value=2.0):
    """Quantizes float32 `features` into string."""
    assert features.dtype == 'float32'
    assert len(features.get_shape()) == 1  # 1-D array
    features = tf.clip_by_value(features, min_quantized_value, max_quantized_value)
    quantize_range = max_quantized_value - min_quantized_value
    features = (features - min_quantized_value) * (255.0 / quantize_range)
    
    features = tf.cast(tf.round(features), dtype=tf.int32, name='feature_pca_qt')
    print(features.get_shape())
    return features

def apply_pca(frame_features):
    """Applies the YouTube8M PCA Transformation over `frame_features`.

    Args:
      frame_features: numpy array of floats, 2048 dimensional vector.

    Returns:
      1024 dimensional vector as a numpy array.
    """
    # Subtract mean
    feats = frame_features - pca_mean

    # Multiply by eigenvectors.
    ## Modify
    feats = tf.reshape(tf.matmul(tf.reshape(feats, [1,2048]), pca_eigenvecs), [1024,])
    # Whiten
    feats /= tf.sqrt(pca_eigenvals + 1e-4)
    return feats

pca_mean, pca_eigenvals, pca_eigenvecs = _load_pca()
f = tf.placeholder(tf.float32,shape=(2048,), name='f')
features_pca = apply_pca(f)
feature_pca_qt = quantize(features_pca)
# feature_pca_qt = tf.clip_by_value(features_pca, min_quantized_value, max_quantized_value)
with tf.Session() as s:
    print(s.run([feature_pca_qt], feed_dict={f: pre_f}))



(1024,)
[array([  0, 218,  97, ..., 177, 134,  40], dtype=int32)]


In [8]:
tf.train.write_graph(tf.get_default_graph(), 'model',
                     'pca_qt_model.pb', as_text=False)

'model/pca_qt_model.pb'

In [9]:
def create_graph(proto_file):
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile(proto_file, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

proto_file = 'model/pca_qt_model.pb'
create_graph(proto_file)
with tf.Session() as s:
    print(s.run([feature_pca_qt], feed_dict={f: pre_f}))

[array([  0, 218,  97, ..., 177, 134,  40], dtype=int32)]


In [10]:
def create_graph_def(proto_file):
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile(proto_file, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        return graph_def
#         [g1_p3] = tf.import_graph_def(graph_def, return_elements=['pool_3/_reshape:0'])

In [11]:
g_def_1 = create_graph_def(proto_file = 'model/classify_image_graph_def.pb')
# g_def_1 = create_graph_def(proto_file = 'model/inception_v3_2016_08_28_frozen.pb')

g_def_2 = create_graph_def(proto_file = 'model/pca_qt_model.pb')
for n in g_def_1.node:
    print(n.name)

DecodeJpeg/contents
DecodeJpeg
Cast
ExpandDims/dim
ExpandDims
ResizeBilinear/size
ResizeBilinear
Sub/y
Sub
Mul/y
Mul
conv/conv2d_params
conv/Conv2D
conv/batchnorm/beta
conv/batchnorm/gamma
conv/batchnorm/moving_mean
conv/batchnorm/moving_variance
conv/batchnorm
conv/CheckNumerics
conv/control_dependency
conv
conv_1/conv2d_params
conv_1/Conv2D
conv_1/batchnorm/beta
conv_1/batchnorm/gamma
conv_1/batchnorm/moving_mean
conv_1/batchnorm/moving_variance
conv_1/batchnorm
conv_1/CheckNumerics
conv_1/control_dependency
conv_1
conv_2/conv2d_params
conv_2/Conv2D
conv_2/batchnorm/beta
conv_2/batchnorm/gamma
conv_2/batchnorm/moving_mean
conv_2/batchnorm/moving_variance
conv_2/batchnorm
conv_2/CheckNumerics
conv_2/control_dependency
conv_2
pool/CheckNumerics
pool/control_dependency
pool
conv_3/conv2d_params
conv_3/Conv2D
conv_3/batchnorm/beta
conv_3/batchnorm/gamma
conv_3/batchnorm/moving_mean
conv_3/batchnorm/moving_variance
conv_3/batchnorm
conv_3/CheckNumerics
conv_3/control_dependency
conv_3
con

In [12]:

image = tf.placeholder(tf.uint8,shape=(100,100,3), name='image')
[g1_p3] = tf.import_graph_def(g_def_1, input_map={'DecodeJpeg:0':image}, return_elements=['pool_3/_reshape:0'])
[g2_feature_pca_qt] = tf.import_graph_def(g_def_2, input_map={'f:0':g1_p3}, return_elements=['feature_pca_qt'])

                                          

In [13]:
# Instantiate extractor. Slow if called first time on your machine, as it
# needs to download 100 MB.
extractor = ft.YouTube8MFeatureExtractor()

image_file = os.path.join(extractor._model_dir, 'cropped_panda.jpg')

im = numpy.array(Image.open(image_file))

In [22]:
import feature_extractor_xs as ft
from PIL import Image
import numpy
import os
tf.reset_default_graph()
image = tf.placeholder(tf.uint8,shape=(100,100,3), name='image')
[g1_p3] = tf.import_graph_def(g_def_1, input_map={'DecodeJpeg:0':image}, 
                              return_elements=['pool_3/_reshape:0'])
[g2_feature_pca_qt] = tf.import_graph_def(g_def_2, input_map={'f:0':g1_p3}, 
return_elements=['feature_pca_qt:0'])

       

with tf.Session() as s:
    print(s.run([g2_feature_pca_qt], feed_dict={'image:0':im}))

[array([  0, 218,  97, ..., 177, 134,  40], dtype=int32)]


In [23]:
tf.train.write_graph(tf.get_default_graph(), 'model',
                     'merge.pb', as_text=False)

'model/merge.pb'

In [26]:
proto_file = 'model/merge.pb'
gd_merge = create_graph(proto_file='model/merge.pb')
# with tf.Session() as s:
#     print(s.run([g2_feature_pca_qt], feed_dict={'image:0':im}))
with tf.Session(graph = tf.get_default_graph()) as s:
    print(s.run([g2_feature_pca_qt], feed_dict={'image:0':im}))

[array([  0, 218,  97, ..., 177, 134,  40], dtype=int32)]
